### Look at pickled data

In [ ]:
import CHReporter

filename = 'OUT/ch_sancheck1H-1resbeta0p8.pkl'
CHReporter.report_saved_data(filename)

### Take a deep look into the saved data

#### 1. Load packages and the file to look into

In [ ]:
import torch as tc
from CommonTools.fileio_tools import save, open_pickled_file
import matplotlib.pyplot as plt
CONST_rcParams = {'font.size' : 16,
                }
plt.rc('font', family='serif')
plt.rcParams.update(CONST_rcParams)

datanames = ("ch_log", "ch_eqn_data", "ch_num_sol", "rep_par")
filenames = ['OUT/ch_sancheck1H-1resbeta0p9.pkl', 
             'OUT/ch_sancheck1H-1resbeta0p8.pkl',
             'OUT/ch_sancheck1H-1resbeta0p7.pkl',
             'OUT/ch_sancheck1H-1resbeta0p6.pkl',
             'OUT/ch_sancheck1H-1resbeta0p5.pkl',
             'OUT/ch_sancheck1H-1resbeta0p4.pkl',
             'OUT/ch_sancheck1H-1resbeta0p3.pkl',
             'OUT/ch_sancheck1H-1resbeta0p2.pkl']
# numer of files
Nfs = len(filenames)

# open files
file = [open_pickled_file(filenames[i]) for i in range(Nfs)]

# unpack main data from each file
# and pack them into a list (to use for-loop for repetitive tasks)
ch_log = [file[i]['ch_log'] for i in range(Nfs)]
ch_eqn_data = [file[i]['ch_eqn_data'] for i in range(Nfs)]
ch_num_sol = [file[i]['ch_num_sol'] for i in range(Nfs)]
rep_par = [file[i]['rep_par'] for i in range(Nfs)]

# # take out data --> lools fine but leads to a weird error
# for dataname in datanames:
#     script = f"{dataname} = [file[i]['{dataname}'] for i in range(Nfs)]"
#     eval(script)

#### 2. Do what you want to do

##### compare their numerical solutions

In [ ]:
# compare their numerical solutions
from CHSolver import plot_still_shot

# create a figure
fig, ax = plt.subplots()

# take out some parameters
Nsaves = rep_par[0]['Nsaves'] # number of saves
print(Nsaves)

# set index of time to look into: integer between 0 and Nsaves (inclusively)
# set also reference and target solution to compare
kk = 20
sol_ref = 0
sol_tar = 7


#=== computer's job 
# find the difference of numerical solutions of reference (sol_ref) and target (sol_tar)
sol_diff = ch_num_sol[sol_tar][kk] - ch_num_sol[sol_ref][kk]
print(f"Maximum difference: {tc.abs(tc.max(sol_diff)):.6f}")
plot_still_shot(fig, ax, ch_eqn_data[0], sol_diff, kk)

##### which one leads to lower energy?

In [ ]:
from CHReporter import plot_seq

# create figure and axes
fig, ax = plt.subplots(figsize=(12,6))

# unpack some parameters to be used
KK_saved = rep_par[0]['last_save_ind'] 
key = 'egy_out' 


#=== Computer's job
# construct legend for each simulation
axes_par = {'legend': [r"$\beta = " + str(ch_eqn_data[i]['beta']) + "$" for i in range(Nfs)]}

# collect energy decay from each simulation
yys = [tc.tensor([ch_log[f][i][key] for i in range(KK_saved)]) for f in range(Nfs)]

# plot them
plot_seq(ax, yys, axes_par=axes_par)

# also print numerical differences of them between the reference simulation (egy_ref) and the other
egy_ref = 0
for jj in range(Nfs):
    print("energy with beta = " + str(ch_eqn_data[jj]['beta'])  + " relative to " + str(ch_eqn_data[egy_ref]['beta']))
    print(yys[jj] - yys[egy_ref])
